In [ ]:
# Extended MNIST (letters and More Digits)

# Importing dependencies
import torch
from PIL import Image
from torch import nn,save,load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# Loading Data

# Loading EMNIST Data (Balanced Split)
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.EMNIST(root="data", split="balanced", download=True, train=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = datasets.EMNIST(root="data", split="balanced", download=True, train=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Define the image classifier model for EMNIST
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 22 * 22, 47)  # EMNIST Balanced has 47 classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [ ]:
# Create an instance of the image classifier model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = ImageClassifier().to(device)

In [ ]:
# Define the optimizer and loss function

from torch.optim import Adam

optimizer = Adam(classifier.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# Train the model
for epoch in range(10):  # Train for 10 epochs
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # Reset gradients
        outputs = classifier(images)  # Forward pass
        loss = loss_fn(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    print(f"Epoch:{epoch} loss is {loss.item()}")

In [ ]:
# Save the trained model
torch.save(classifier.state_dict(), 'model_state.pt')

In [ ]:
# Load the saved model
with open('model_state.pt', 'rb') as f: 
     classifier.load_state_dict(load(f))  

In [ ]:
# Perform inference on an image
img = Image.open('image.jpg')
img_transform = transforms.Compose([transforms.ToTensor()])
img_tensor = img_transform(img).unsqueeze(0).to(device)
output = classifier(img_tensor)
predicted_label = torch.argmax(output)
print(f"Predicted label: {predicted_label}")

In [ ]:
# EMNIST Balanced label-to-character mapping

emnist_classes = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
    'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
    'U', 'V', 'W', 'X', 'Y', 'Z',
    'a', 'b', 'd', 'e', 'f', 'g', 'h', 'n', 'q', 'r', 't'
]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Visualize Predictions with Matplotlib

#Downgrade NUmPy and upgrade dependencies (run once, then restart kernel)

!pip install "numpy<2"
!pip install --upgrade matplotlib
!pip install --upgrade pybind11

# Show 5 random test samples with predictions
classifier.eval()
with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        outputs = classifier(images)
        predicted = torch.argmax(outputs, dim=1)

        # Plot first 5 images
        for j in range(5):
            img = images[j].cpu().squeeze(0)  # remove channel dim
            label = emnist_classes[predicted[j].item()]
            plt.subplot(1, 5, j+1)
            plt.imshow(img, cmap='gray')
            plt.title(f'Pred: {label}')
            plt.axis('off')
        plt.show()
        break  # only show one batch